In [9]:
# Sentinel API code. Import and install sentinelsat package if not already
# import sys
# !{sys.executable} -m pip install sentinelsat
# Then import required packages
# SentinelAPI - Class to connect to Copernicus Open Access Hub, search and download imagery.
# read_geojson - Import a geojson file for use.
# geojson_to_wkt - Convert a GeoJSON object to Well-Known Text. 
    # Intended for use with OpenSearch queries. 3D points are converted to 2D.
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [10]:
# Create API object with username/password
api = SentinelAPI(
    'akerr8', 
    'Uncanny1@1', 
    'https://apihub.copernicus.eu/apihub'
)
api

In [11]:
# Create geojson
my_geojson = {
  "type": "Feature", # define GeoJSON type. As it's a single geometry, it's just a single 'Feature'
  "geometry": {
	"type": "Point", # define the geometry type, which is a point
	"coordinates": [ # Here are the actual geometry coordinates
		  67.99441034239601,
		  27.596445925557646
		],
  },
    "properties": {'id': 'WHF'}, # Geometry ID
}
my_geojson
footprint = geojson_to_wkt(my_geojson) # Convert Well Known Text (WKT) format
footprint

'POINT(67.9944 27.5964)'

In [15]:
# Set API query by point, time, and SciHub query keywords
image_metadata = api.query(
    footprint,
    date=('20220601', '20220930'), # set dates(2015, 12, 29)
    platformname='Sentinel-2'                    
)
image_metadata
len(image_metadata) # how many images

Querying products:  52%|#####2    | 100/192 [00:00<?, ?product/s]

192

In [13]:
# Save to a dataframe
import pandas as pd
image_metadata_df = api.to_dataframe(image_metadata)
image_metadata_df.to_csv('metadata.csv')

In [14]:
# sort data based on the cloud cover of each image, and the date the image was ingested
image_metadata_df_sorted = image_metadata_df.sort_values(
    ['cloudcoverpercentage'], ascending=[True])
image_metadata_df_sorted.to_csv('metadata_sorted.csv')

In [27]:
#subset using .head function
image_metadata_df_sorted = image_metadata_df_sorted.head(1)
len(image_metadata_df_sorted)

1

In [28]:
api.download_all(image_metadata_df_sorted.index) # download the image

MD5 checksumming:   0%|          | 0.00/829M [00:00<?, ?B/s]

ResultTuple(downloaded={'8469a990-1a8a-4bb6-b99a-1591c641f2fa': {'id': '8469a990-1a8a-4bb6-b99a-1591c641f2fa', 'title': 'S2A_MSIL1C_20221007T155141_N0400_R054_T18TWL_20221007T205840', 'size': 828868922, 'md5': '0adfcf38402471f70d3b0a0c633661fc', 'date': datetime.datetime(2022, 10, 7, 15, 51, 41, 24000), 'footprint': 'POLYGON((-73.69681 40.886514353249446,-73.70148 40.87369590327607,-73.755035 40.72744370814638,-73.808105 40.58107975911195,-73.81711 40.556111425058205,-75.000244 40.56274991351036,-75.000244 41.551844670066835,-73.68381 41.54431153543986,-73.69681 40.886514353249446))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('8469a990-1a8a-4bb6-b99a-1591c641f2fa')/$value", 'Online': True, 'Creation Date': datetime.datetime(2022, 10, 7, 23, 29, 0, 801000), 'Ingestion Date': datetime.datetime(2022, 10, 7, 23, 28, 0, 611000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('8469a990-1a8a-4bb6-b99a-1591c641f2fa')/Products('Quicklook')/$value", '